# SUMMARY

In [ ]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [ ]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
!pip install --upgrade datasets transformers accelerate evaluate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 165.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 100.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0

In [ ]:
!pip install evaluate rouge_score transformers[torch]

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dd72abd746674a1ae6a6c58dc2fc75d4a45a562952c11b0f71c6e0db0978c33f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Importing libraries for data preparation and visualization
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing warnings to ignore them
import warnings
warnings.filterwarnings("ignore")

# Importing libraries for pickle, json, and joblib files
import pickle
import json
import joblib
import copy

# Importing train_test_split for data splitting
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset, DatasetDict, concatenate_datasets, Dataset

# SUMMARY

In [ ]:
from transformers import BertTokenizer, GPT2Tokenizer, EncoderDecoderModel
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, EarlyStoppingCallback
from datasets import Dataset, DatasetDict, load_metric
import evaluate
import torch
import os
from os import listdir
from os.path import isfile, join
import json
import re
import numpy as np
import pandas as pd

In [ ]:
from datasets import load_dataset

ds_edin = load_dataset("EdinburghNLP/xsum")

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

ds_gopal = load_dataset("gopalkalpande/bbc-news-summary")

Generating train split:   0%|          | 0/2224 [00:00<?, ? examples/s]

In [ ]:
ds_edin

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [ ]:
small_train_dataset = ds_edin['train'].shuffle(seed=42).select([i for i in list(range(5000))])
small_test_dataset = ds_edin['test'].shuffle(seed=42).select([i for i in list(range(300))])
small_valid_dataset = ds_edin['validation'].shuffle(seed=42).select([i for i in list(range(300))])

In [ ]:
dataset = DatasetDict({
    'train': small_train_dataset,
    'test': small_test_dataset,
    'valid':small_valid_dataset})
# see structure
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 300
    })
    valid: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 300
    })
})

In [ ]:
dataset_run = dataset.remove_columns(["id"])

In [ ]:
dataset_run

DatasetDict({
    train: Dataset({
        features: ['document', 'summary'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['document', 'summary'],
        num_rows: 300
    })
    valid: Dataset({
        features: ['document', 'summary'],
        num_rows: 300
    })
})

In [ ]:
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization")

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
prefix = ""
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset_run.map(preprocess_function, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
training_args = Seq2SeqTrainingArguments(
    output_dir="./bert2bertnews",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="rouge1",  # Metric to use for early stopping
    greater_is_better=True,  # Whether a higher value of the metric is better
    save_steps=500,  # Define the number of steps after which to save the model
    eval_steps=500,  # Define the number of steps after which to run evaluation
    push_to_hub=True
)

In [ ]:
rouge = evaluate.load('rouge')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    ### BOLEH DIGANTI SESUAI KEBUTUHAN MODEL ###
    # if isinstance(predictions, tuple):
    #     predictions = predictions[0]
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    ### ###

    # Directly decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=False)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=False)

    # Rouge expects newline-separated text
    decoded_preds = ["\n".join(decoded_pred.split()) for decoded_pred in decoded_preds]
    decoded_labels = ["\n".join(decoded_label.split()) for decoded_label in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['document', 'summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
    valid: Dataset({
        features: ['document', 'summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 300
    })
})

In [ ]:
# BOLEH DIGANTI SESUAI KEBUTUHAN MODEL
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    compute_metrics=compute_metrics,
    data_collator=data_collator
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,4.137300,3.898204,0.374300,0.250500,0.346500,0.374400
2,3.585500,3.731229,0.356300,0.228600,0.327500,0.356300
3,3.171900,3.676628,0.357700,0.229100,0.329200,0.357900
4,2.898700,3.649930,0.339100,0.208600,0.310200,0.339600
5,2.754700,3.655338,0.319300,0.186400,0.290300,0.319600
6,2.498700,3.683530,0.335400,0.204200,0.307900,0.335800
7,2.311400,3.737063,0.343000,0.211300,0.313200,0.343600
8,2.231700,3.768001,0.320900,0.186200,0.288700,0.321200
9,2.162000,3.778511,0.322100,0.190300,0.292900,0.322400
10,2.033700,3.793441,0.321500,0.187100,0.289900,0.321800


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 40, 'min_length': 20, 'early_stopping': True, 'num_beams': 10, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}
Some non-default generation parameters are set in the model config. These should go into a Gener

TrainOutput(global_step=1570, training_loss=2.819604127725978, metrics={'train_runtime': 2624.9175, 'train_samples_per_second': 19.048, 'train_steps_per_second': 0.598, 'total_flos': 1.53364660224e+16, 'train_loss': 2.819604127725978, 'epoch': 10.0})

In [ ]:
tokenizer.push_to_hub("bert2bertnews")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/anggari/bert2bertnews/commit/92c44d461a8f52e46f1593fb1207a003d6700d56', commit_message='Upload tokenizer', commit_description='', oid='92c44d461a8f52e46f1593fb1207a003d6700d56', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    # Definisikan pola regex untuk berbagai pembersihan
    url_pattern = re.compile(r'https?://\S+|www\.\S+', re.IGNORECASE)
    hashtag_pattern = re.compile(r'#\w+', re.IGNORECASE)
    double_space_pattern = re.compile(r'\s\s+')
    header_pattern = re.compile(r'^.*?--\s?', re.IGNORECASE)
    video_pattern = re.compile(r'VIDEO:.*?(?:\.\s|$)', re.IGNORECASE)

    # Hapus URL
    text = url_pattern.sub('', text)

    # Hapus hashtag
    text = hashtag_pattern.sub('', text)

    # Cek jika ada '--' dalam 40 karakter pertama
    if '--' in text[:40]:
        # Hapus header sebelum '--'
        text = header_pattern.sub('', text).strip()

    # Hapus frasa "VIDEO:" hingga titik
    text = video_pattern.sub('', text)

    # Hapus double space
    text = double_space_pattern.sub(' ', text)

    # Trim leading and trailing spaces
    text = text.strip()

    return text

In [ ]:
def inference(url):
  response = requests.get(url)
  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    select = soup.get_text().strip().rstrip("\n").replace("\n", "")
    text = " ".join(select.split())
  else:
    text="Failed"
  words = text.split()
  # Gunakan clean_text untuk membersihkan konteks
  cleaned_text = clean_text(text)
  # Trim to the first 1000 words
  trimmed_text = ' '.join(words[:100])
  sentiment = sentiment_model([trimmed_text])
  input_ids = tokenizer.encode(trimmed_text, return_tensors='pt')
  summary_ids = model.generate(input_ids.to(model.device),
            min_length=20,
            max_length=128,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

  summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return cleaned_text, sentiment, summary_text



In [ ]:
sentiment_model = pipeline(model="anggari/distil_news_finetune")
model = EncoderDecoderModel.from_pretrained("anggari/bert2bertnews")
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


config.json:   0%|          | 0.00/4.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/999M [00:00<?, ?B/s]

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

In [ ]:
start_time = time.time()
text, sentiment, summary_text = inference("https://www.bbc.com/news/articles/cqxjn3w9228o")
end_time = time.time()
inference_time = end_time - start_time


In [ ]:
print(f'text : {text} \nsentiment : {sentiment} \nsummary:{summary_text}\ninference time is {inference_time}')

text : Typhoon Yagi collapses busy bridge in VietnamSkip to contentBritish Broadcasting CorporationWatchHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS ElectionUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC InDepthBBC VerifySportBusinessExecutive LoungeTechnology of BusinessWomen at the HelmFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral AmericaEuropeMiddle EastNorth AmericaSouth AmericaWorld’s TableCulture & ExperiencesAdventuresThe SpeciaListEarthNatural WondersWeather & ScienceClimate SolutionsSustainable BusinessGreen LivingVideoLiveLive NewsLive SportHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveAudioWeath

In [ ]:
text

"Typhoon Yagi collapses busy bridge in VietnamSkip to contentBritish Broadcasting CorporationWatchHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveHomeNewsIsrael-Gaza WarWar in UkraineUS ElectionUS & CanadaUKUK PoliticsEnglandN. IrelandN. Ireland PoliticsScotlandScotland PoliticsWalesWales PoliticsAfricaAsiaChinaIndiaAustraliaEuropeLatin AmericaMiddle EastIn PicturesBBC InDepthBBC VerifySportBusinessExecutive LoungeTechnology of BusinessWomen at the HelmFuture of BusinessInnovationTechnologyScience & HealthArtificial IntelligenceAI v the MindCultureFilm & TVMusicArt & DesignStyleBooksEntertainment NewsTravelDestinationsAfricaAntarcticaAsiaAustralia and PacificCaribbean & BermudaCentral AmericaEuropeMiddle EastNorth AmericaSouth AmericaWorld’s TableCulture & ExperiencesAdventuresThe SpeciaListEarthNatural WondersWeather & ScienceClimate SolutionsSustainable BusinessGreen LivingVideoLiveLive NewsLive SportHomeNewsSportBusinessInnovationCultureTravelEarthVideoLiveAudioWeatherNews